# NLP Demos
## Setup
Make sure that you install the `transformers` and `bertviz` libraries. You may also need to install a backend ML support library like PyTorch, Keras, or TensorFlow.

This demo is heavily based on the walkthroughs and demos provided by the author of bertviz at its [repository](https://github.com/jessevig/bertviz).

In [ ]:
!pip install transformers
!pip install bertviz

## Importing BERT
Most models that you will use for basic tasks will come pretrained from repositories such as HuggingFace. The `transformers` package does a great job automatically downloading and setting up models for you. Notice that we set `output_attentions` to true so that we can store extra data about model runs for visualization.

In [ ]:
from transformers import AutoTokenizer, AutoModel, utils

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased", output_attentions=True)

## Running the model on a sentence

In [ ]:
sentence = "The tired fox wrapped itself in a blanket to keep warm."

inputs = tokenizer.encode(sentence, return_tensors='pt') # encode sentence using tokenizer
outputs = model(inputs) # run model on tokens
attention = outputs[-1] # take the attentions from the output
tokens = tokenizer.convert_ids_to_tokens(inputs[0])  # store representation of tokens

## Basic visualization of attention

See [part 1](https://towardsdatascience.com/deconstructing-bert-distilling-6-patterns-from-100-million-parameters-b49113672f77) and [part 2](https://towardsdatascience.com/deconstructing-bert-part-2-visualizing-the-inner-workings-of-attention-60a16d86b5c1) by the author of BertViz to get a sense for what's going on.

In [ ]:
from bertviz import model_view

model_view(attention, tokens)

In [ ]:
from bertviz import head_view

head_view(attention, tokens)

## Neuron view of BERT

In [ ]:
from bertviz.transformers_neuron_view import BertModel, BertTokenizer
from bertviz.neuron_view import show

model_type = 'bert'
model_version = 'bert-base-uncased'
do_lower_case = True
sentence_a = "The fox sat on a blanket to keep warm."
sentence_b = "The fox lay on a blanket to stay warm"
model = BertModel.from_pretrained(model_version, output_attentions=True)
tokenizer = BertTokenizer.from_pretrained(model_version, do_lower_case=do_lower_case)
show(model, model_type, tokenizer, sentence_a, sentence_b, display_mode='light', layer=2, head=0)

## Comparing with GPT-2
Recall the architectural differences we discussed between BERT and GPT!

In [ ]:
from bertviz.transformers_neuron_view import GPT2Model, GPT2Tokenizer
from bertviz.neuron_view import show

model_type = 'gpt2'
model_version = 'gpt2'
model = GPT2Model.from_pretrained(model_version)
tokenizer = GPT2Tokenizer.from_pretrained(model_version)
text = "At the store, she bought apples, oranges, and pineapple,"
show(model, model_type, tokenizer, text, display_mode='light')

## Examining an encoder-decoder setup
Encoder-decoder models work as you might guess from the name. First, the model encodes some text into a vector representation. Then, the decoder takes that representation and turns it back into text again (usually in a different form). Here we'll use the MarianMT model series for english to french translation.

In [ ]:
from transformers import AutoTokenizer, AutoModel

# Load model
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-fr")
model = AutoModel.from_pretrained("Helsinki-NLP/opus-mt-en-fr", output_attentions=True)

# Encode and decode some text
encoder_input_ids = tokenizer("My dog doesn't have a nose.", return_tensors="pt", add_special_tokens=True).input_ids
with tokenizer.as_target_tokenizer():
    decoder_input_ids = tokenizer("Mon chien n'a pas de nez.", return_tensors="pt", add_special_tokens=True).input_ids

outputs = model(input_ids=encoder_input_ids, decoder_input_ids=decoder_input_ids)

encoder_text = tokenizer.convert_ids_to_tokens(encoder_input_ids[0])
decoder_text = tokenizer.convert_ids_to_tokens(decoder_input_ids[0])

from bertviz import model_view
model_view(
    encoder_attention=outputs.encoder_attentions,
    decoder_attention=outputs.decoder_attentions,
    cross_attention=outputs.cross_attentions,
    encoder_tokens= encoder_text,
    decoder_tokens = decoder_text
)

Now one more, this time for BART fine-tuned for summarization tasks.

In [ ]:
from transformers import AutoTokenizer, AutoModel, utils
from bertviz import model_view

# Load BART fine-tuned for summarization on CNN/Daily Mail dataset
model_name = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name, output_attentions=True)

# get encoded input vectors
encoder_input_ids = tokenizer("Mayor Eric Adams is amassing a team of high-powered lawyers paid by city taxpayer and donor money.", return_tensors="pt", add_special_tokens=True).input_ids

# create ids of encoded input vectors
decoder_input_ids = tokenizer("Mayor Adams builds team of lawyers using taxpayer money.", return_tensors="pt", add_special_tokens=True).input_ids

outputs = model(input_ids=encoder_input_ids, decoder_input_ids=decoder_input_ids)

encoder_text = tokenizer.convert_ids_to_tokens(encoder_input_ids[0])
decoder_text = tokenizer.convert_ids_to_tokens(decoder_input_ids[0])

model_view(
    encoder_attention=outputs.encoder_attentions,
    decoder_attention=outputs.decoder_attentions,
    cross_attention=outputs.cross_attentions,
    encoder_tokens= encoder_text,
    decoder_tokens=decoder_text
)

## More things to try

- The [Ecco](https://github.com/jalammar/ecco/) library is also very interesting, but can be very finicky to set up (especially on arm64 Macs). You can see some of their examples [here](https://ecco.readthedocs.io/en/main/)
- For something even more fully featured, check out [LIT](https://github.com/PAIR-code/lit)
- HuggingFace also hosts the [Exbert](https://huggingface.co/spaces/exbert-project/exbert) tool

### LIT demos

After installing, try:
- `python -m lit_nlp.examples.glue_demo --quickstart --port=5431`
- `python -m lit_nlp.examples.image_demo --port=5431`
- `python -m lit_nlp.examples.toxicity_demo --port=5431`

Navigate to [localhost:5431](http://localhost:5431)